In [16]:
import pandas as pd
import numpy as np

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine
import clickhouse_connect
from sqlalchemy.orm import Session
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

import os
import datetime
from urllib.parse import quote_plus

In [18]:
class DataWarehouse:
    DB_ADDRESS = os.getenv('DWH_HOST', '35.77.20.66')
    DB_PORT = os.getenv('DWH_PORT', '8123')
    DB_USER = os.getenv('DWH_USER', 'dwh_db')
    DB_PASSWORD = quote_plus(os.getenv('DWH_PASSWORD', 'dwh_db'))
    DB_NAME = os.getenv('DWH_DB', 'dwh_db')

    DB_URL = f'clickhouse://{DB_USER}:{DB_PASSWORD}@{DB_ADDRESS}/{DB_NAME}'

    @classmethod
    def get_client(cls):
        return clickhouse_connect.get_client(
            host=cls.DB_ADDRESS,
            port=cls.DB_PORT,
            username=cls.DB_USER,
            password=cls.DB_PASSWORD,
            database=cls.DB_NAME
        )

    @classmethod
    def get_session(cls) -> Session:
        s = Session(cls.get_engine())
        try:
            yield s
        finally:
            s.close()

    @classmethod
    def get_engine(cls) -> Engine:
        return create_engine(cls.get_uri())

    @classmethod
    def get_uri(cls) -> str:
        return cls.DB_URL
    
    @classmethod
    def read(cls, query, *args, **kwargs):
        return cls.get_client().query_df(query)

In [19]:
client = DataWarehouse()

In [34]:
Q = '''
    select
        h_ticker_name,
        h_exchange_name,
        max(pit_price) as high,
        min(pit_price) as low,
        argMin(pit_price, pit_ts) as open,
        argMax(pit_price, pit_ts) as close,
        pit_ts
    from 
        pit_big_table_whole_market_trades_history
    where
        h_ticker_name = 'MATICUSDT' and
        h_exchange_name = 'binance_usdt_m' and
        pit_ts > '2023-03-08 00:00:00' and
        pit_ts < '2023-03-09 00:00:00'
    group by
        toStartOfSecond(toDateTime64(pit_ts, 3)) as pit_ts,
        h_ticker_name,
        h_exchange_name
    order by
        pit_ts desc
'''

In [35]:
cex = client.read(Q)
cex.tail()

,h_ticker_name,h_exchange_name,high,low,open,close,pit_ts
68444,MATICUSDT,binance_usdt_m,1.1498,1.1497,1.1498,1.1498,2023-03-08 00:00:09
68445,MATICUSDT,binance_usdt_m,1.1497,1.1494,1.1497,1.1497,2023-03-08 00:00:08
68446,MATICUSDT,binance_usdt_m,1.1496,1.1494,1.1495,1.1495,2023-03-08 00:00:07
68447,MATICUSDT,binance_usdt_m,1.1498,1.1497,1.1497,1.1497,2023-03-08 00:00:05
68448,MATICUSDT,binance_usdt_m,1.1500,1.1496,1.1498,1.1498,2023-03-08 00:00:04


In [39]:
cex.rename(
    columns={
        'pit_ts': 'ts'
    },
    inplace=True
)

In [26]:
Q = '''
    with agg as (
        select 
            *,
            addMilliseconds(toDateTime64(pit_ts, 3), pit_index_position_in_the_block) as ts
        from
          pit_big_table_bids_and_asks
        WHERE 
          ts BETWEEN '2023-03-08 00:00:00' and '2023-03-09 00:00:00' and
          h_protocol_name = 'uniswapV3' and
          pit_symbol = 'WMATIC/USDT'
        order by
          pit_ts asc
    ) select
        pit_symbol,
        h_protocol_name,
        max(pit_price) as high,
        min(pit_price) as low,
        argMin(pit_price, ts) as open,
        argMax(pit_price, ts) as close,
        ts
    from 
        agg
    group by
        toStartOfSecond(toDateTime64(ts, 3)) as ts,
        h_protocol_name,
        pit_symbol
    order by
        ts asc
'''

# №1

In [21]:
quickswapV3_wmatic_usdc_df = client.read(Q)
quickswapV3_wmatic_usdc_df.head()

,pit_symbol,h_protocol_name,high,low,open,close,ts
0,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,2023-03-08 00:00:18
1,WMATIC/USDC,quickswapV3,1.149030,1.149030,1.149030,1.149030,2023-03-08 00:00:46
2,WMATIC/USDC,quickswapV3,1.150201,1.150201,1.150201,1.150201,2023-03-08 00:00:56
3,WMATIC/USDC,quickswapV3,1.150281,1.150281,1.150281,1.150281,2023-03-08 00:01:12
4,WMATIC/USDC,quickswapV3,1.150789,1.150530,1.150530,1.150530,2023-03-08 00:01:14


In [22]:
quickswapV3_wmatic_usdc_df.shape

(3387, 7)

In [29]:
quickswapV3_wmatic_usdc_df.rename(
    columns={
        'high': 'quickswapV3_wmatic_usdc_high',
        'low': 'quickswapV3_wmatic_usdc_low',
        'open': 'quickswapV3_wmatic_usdc_open',
        'close': 'quickswapV3_wmatic_usdc_close'
    },
    inplace=True
)

# №2

In [24]:
uniswapV3_wmatic_usdc_df = client.read(Q)
uniswapV3_wmatic_usdc_df.head()

,pit_symbol,h_protocol_name,high,low,open,close,ts
0,WMATIC/USDC,uniswapV3,1.149746,1.149746,1.149746,1.149746,2023-03-08 00:00:02
1,WMATIC/USDC,uniswapV3,1.149806,1.149806,1.149806,1.149806,2023-03-08 00:00:06
2,WMATIC/USDC,uniswapV3,1.149809,1.149809,1.149809,1.149809,2023-03-08 00:00:14
3,WMATIC/USDC,uniswapV3,1.149916,1.149916,1.149916,1.149916,2023-03-08 00:00:16
4,WMATIC/USDC,uniswapV3,1.150048,1.150048,1.150048,1.150048,2023-03-08 00:00:18


In [25]:
uniswapV3_wmatic_usdc_df.shape

(10194, 7)

In [30]:
uniswapV3_wmatic_usdc_df.rename(
    columns={
        'high': 'uniswapV3_wmatic_usdc_high',
        'low': 'uniswapV3_wmatic_usdc_low',
        'open': 'uniswapV3_wmatic_usdc_open',
        'close': 'uniswapV3_wmatic_usdc_close'
    },
    inplace=True
)

# №3

In [27]:
uniswapV3_wmatic_usdt_df = client.read(Q)
uniswapV3_wmatic_usdt_df.head()

,pit_symbol,h_protocol_name,high,low,open,close,ts
0,WMATIC/USDT,uniswapV3,1.150331,1.150215,1.150215,1.150215,2023-03-08 00:00:18
1,WMATIC/USDT,uniswapV3,1.150425,1.150425,1.150425,1.150425,2023-03-08 00:00:22
2,WMATIC/USDT,uniswapV3,1.150507,1.150507,1.150507,1.150507,2023-03-08 00:00:24
3,WMATIC/USDT,uniswapV3,1.149415,1.149413,1.149413,1.149413,2023-03-08 00:00:36
4,WMATIC/USDT,uniswapV3,1.150624,1.150624,1.150624,1.150624,2023-03-08 00:00:56


In [28]:
uniswapV3_wmatic_usdt_df.shape

(6982, 7)

In [31]:
uniswapV3_wmatic_usdt_df.rename(
    columns={
        'high': 'uniswapV3_wmatic_usdt_high',
        'low': 'uniswapV3_wmatic_usdt_low',
        'open': 'uniswapV3_wmatic_usdt_open',
        'close': 'uniswapV3_wmatic_usdt_close'
    },
    inplace=True
)

## MERGE

In [36]:
df = uniswapV3_wmatic_usdc_df.merge(quickswapV3_wmatic_usdc_df, how='outer', on='ts').sort_values('ts').fillna(method='ffill').dropna()
df.head()

,pit_symbol_x,h_protocol_name_x,uniswapV3_wmatic_usdc_high,uniswapV3_wmatic_usdc_low,uniswapV3_wmatic_usdc_open,uniswapV3_wmatic_usdc_close,ts,pit_symbol_y,h_protocol_name_y,quickswapV3_wmatic_usdc_high,quickswapV3_wmatic_usdc_low,quickswapV3_wmatic_usdc_open,quickswapV3_wmatic_usdc_close
4,WMATIC/USDC,uniswapV3,1.150048,1.150048,1.150048,1.150048,2023-03-08 00:00:18,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016
5,WMATIC/USDC,uniswapV3,1.150227,1.150227,1.150227,1.150227,2023-03-08 00:00:20,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016
6,WMATIC/USDC,uniswapV3,1.149226,1.149226,1.149226,1.149226,2023-03-08 00:00:22,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016
7,WMATIC/USDC,uniswapV3,1.150474,1.149422,1.150474,1.150474,2023-03-08 00:00:24,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016
8,WMATIC/USDC,uniswapV3,1.149421,1.149421,1.149421,1.149421,2023-03-08 00:00:40,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016


In [38]:
df = df.merge(uniswapV3_wmatic_usdt_df, how='outer', on='ts').sort_values('ts').fillna(method='ffill').dropna()
df.head()

,pit_symbol_x,h_protocol_name_x,uniswapV3_wmatic_usdc_high,uniswapV3_wmatic_usdc_low,uniswapV3_wmatic_usdc_open,uniswapV3_wmatic_usdc_close,ts,pit_symbol_y,h_protocol_name_y,quickswapV3_wmatic_usdc_high,quickswapV3_wmatic_usdc_low,quickswapV3_wmatic_usdc_open,quickswapV3_wmatic_usdc_close,pit_symbol,h_protocol_name,uniswapV3_wmatic_usdt_high,uniswapV3_wmatic_usdt_low,uniswapV3_wmatic_usdt_open,uniswapV3_wmatic_usdt_close
0,WMATIC/USDC,uniswapV3,1.150048,1.150048,1.150048,1.150048,2023-03-08 00:00:18,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,WMATIC/USDT,uniswapV3,1.150331,1.150215,1.150215,1.150215
1,WMATIC/USDC,uniswapV3,1.150227,1.150227,1.150227,1.150227,2023-03-08 00:00:20,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,WMATIC/USDT,uniswapV3,1.150331,1.150215,1.150215,1.150215
2,WMATIC/USDC,uniswapV3,1.149226,1.149226,1.149226,1.149226,2023-03-08 00:00:22,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,WMATIC/USDT,uniswapV3,1.150425,1.150425,1.150425,1.150425
3,WMATIC/USDC,uniswapV3,1.150474,1.149422,1.150474,1.150474,2023-03-08 00:00:24,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,WMATIC/USDT,uniswapV3,1.150507,1.150507,1.150507,1.150507
11650,WMATIC/USDC,uniswapV3,1.150474,1.149422,1.150474,1.150474,2023-03-08 00:00:36,WMATIC/USDC,quickswapV3,1.150016,1.150016,1.150016,1.150016,WMATIC/USDT,uniswapV3,1.149415,1.149413,1.149413,1.149413


In [40]:
df = df.merge(cex, how='outer', on='ts').sort_values('ts').fillna(method='ffill').dropna()
df.head()

,pit_symbol_x,h_protocol_name_x,uniswapV3_wmatic_usdc_high,uniswapV3_wmatic_usdc_low,uniswapV3_wmatic_usdc_open,uniswapV3_wmatic_usdc_close,ts,pit_symbol_y,h_protocol_name_y,quickswapV3_wmatic_usdc_high,...,uniswapV3_wmatic_usdt_high,uniswapV3_wmatic_usdt_low,uniswapV3_wmatic_usdt_open,uniswapV3_wmatic_usdt_close,h_ticker_name,h_exchange_name,high,low,open,close
0,WMATIC/USDC,uniswapV3,1.150048,1.150048,1.150048,1.150048,2023-03-08 00:00:18,WMATIC/USDC,quickswapV3,1.150016,...,1.150331,1.150215,1.150215,1.150215,MATICUSDT,binance_usdt_m,1.1506,1.1503,1.1505,1.1505
70501,WMATIC/USDC,uniswapV3,1.150048,1.150048,1.150048,1.150048,2023-03-08 00:00:19,WMATIC/USDC,quickswapV3,1.150016,...,1.150331,1.150215,1.150215,1.150215,MATICUSDT,binance_usdt_m,1.1506,1.1506,1.1506,1.1506
1,WMATIC/USDC,uniswapV3,1.150227,1.150227,1.150227,1.150227,2023-03-08 00:00:20,WMATIC/USDC,quickswapV3,1.150016,...,1.150331,1.150215,1.150215,1.150215,MATICUSDT,binance_usdt_m,1.1506,1.1505,1.1505,1.1505
70500,WMATIC/USDC,uniswapV3,1.150227,1.150227,1.150227,1.150227,2023-03-08 00:00:21,WMATIC/USDC,quickswapV3,1.150016,...,1.150331,1.150215,1.150215,1.150215,MATICUSDT,binance_usdt_m,1.1505,1.1502,1.1503,1.1503
2,WMATIC/USDC,uniswapV3,1.149226,1.149226,1.149226,1.149226,2023-03-08 00:00:22,WMATIC/USDC,quickswapV3,1.150016,...,1.150425,1.150425,1.150425,1.150425,MATICUSDT,binance_usdt_m,1.1503,1.1503,1.1503,1.1503


In [59]:
df.to_csv('c3d3_ohlc.csv')

In [58]:
df.shape

(70499, 25)

In [57]:
df = df[df['uniswapV3_wmatic_usdt_close'] != 0]